In [1]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()
import json
import numpy as np
import requests
from config import gkey

In [2]:
#create engine and import data from sqlite database
engine = create_engine('sqlite:///../static/data/bredat.sqlite', echo=False)
Base.metadata.create_all(engine)
Base.prepare(engine, reflect=True)
session = Session(engine)

In [3]:
#create dataframe
brew_df = pd.read_sql_query("SELECT * FROM business", con=engine)
brew_df

,index,CompanyName,street,City,State,ZipCode,ZipPlus,ContactFirstName,ContactLastName,ContactTitle,...,NAICS2,NAICS3,NAICS4,NAICS5,BusinessType,County,Employee,AnnualSales,URL,YearEstablished
0,0,217 Brew Works,217 South St S,Wilson,NC,27893,4911.0,Tom,Curran,Owner,...,NaN,NaN,NaN,NaN,Private,Wilson,14,10453000.0,217BREWWORKS.COM,2016
1,1,All Sevens Brewing,777 Haywood Rd,Asheville,NC,28806,3132.0,None,None,None,...,NaN,NaN,NaN,NaN,Private,Buncombe,23,34511000.0,None,2018
2,2,Altamont Brewing Co,1050 Haywood Rd,Asheville,NC,28806,2639.0,None,None,None,...,NaN,NaN,NaN,NaN,Private,Buncombe,23,34511000.0,None,2017
3,3,Andrews Brewing Co,1026 Main St,Andrews,NC,28901,NaN,None,None,None,...,NaN,NaN,NaN,NaN,Private,Cherokee,14,17916000.0,None,2018
4,4,Angry Troll Brewing,222 E Main St # U2,Elkin,NC,28621,3493.0,Cicely,Mc Culloch,Owner,...,NaN,NaN,NaN,NaN,Private,Surry,12,16648000.0,ANGRYTROLLBREWING.COM,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,271,Wise Man Brewing,826 Angelos Bros Ave,Winston Salem,NC,27101,1524.0,Michael D,Beverly,Manager,...,NaN,NaN,NaN,NaN,Private,Forsyth,14,5870000.0,WISEMANBREWING.COM,2017
272,272,Wooden Robot Brewery LLC,1440 S Tryon St # 110,Charlotte,NC,28203,4263.0,Joshua,Patton,CEO,...,NaN,NaN,NaN,NaN,Private,Mecklenburg,14,7049000.0,WOODENROBOTBREWERY.COM,2015
273,273,Yesteryears Brewery,300 E Main St # C,Carrboro,NC,27510,2359.0,David,Larsen,Manager,...,NaN,NaN,NaN,NaN,Private,Orange,14,21819000.0,None,2016
274,274,York Chester Brewing Co,1500 River Dr # 250,Belmont,NC,28012,3587.0,None,None,None,...,NaN,NaN,NaN,NaN,Private,Gaston,14,20830000.0,YORKCHESTERBREW.COM,2017


In [4]:
# get latitude and longitude
brew_df['latitude'] = ''
brew_df['longitude'] = ''
base_url = 'https://maps.googleapis.com/maps/api/geocode/json'
for index, row in brew_df.iterrows():
    address = f"{row.street}, {row.City}, {row.State}"
    params = {
        "address": address,
        "key": gkey
    }

    response = requests.get(base_url, params=params)
    results = response.json()["results"][0]['geometry']['location']

    try:
        brew_df.at[index, 'latitude'] = results["lat"]
        brew_df.at[index, 'longitude'] = results["lng"]
    except:
        pass

In [5]:
brew_df

,index,CompanyName,street,City,State,ZipCode,ZipPlus,ContactFirstName,ContactLastName,ContactTitle,...,NAICS4,NAICS5,BusinessType,County,Employee,AnnualSales,URL,YearEstablished,latitude,longitude
0,0,217 Brew Works,217 South St S,Wilson,NC,27893,4911.0,Tom,Curran,Owner,...,NaN,NaN,Private,Wilson,14,10453000.0,217BREWWORKS.COM,2016,35.7229,-77.9121
1,1,All Sevens Brewing,777 Haywood Rd,Asheville,NC,28806,3132.0,None,None,None,...,NaN,NaN,Private,Buncombe,23,34511000.0,None,2018,35.5787,-82.5943
2,2,Altamont Brewing Co,1050 Haywood Rd,Asheville,NC,28806,2639.0,None,None,None,...,NaN,NaN,Private,Buncombe,23,34511000.0,None,2017,35.5815,-82.6044
3,3,Andrews Brewing Co,1026 Main St,Andrews,NC,28901,NaN,None,None,None,...,NaN,NaN,Private,Cherokee,14,17916000.0,None,2018,35.2008,-83.8244
4,4,Angry Troll Brewing,222 E Main St # U2,Elkin,NC,28621,3493.0,Cicely,Mc Culloch,Owner,...,NaN,NaN,Private,Surry,12,16648000.0,ANGRYTROLLBREWING.COM,2015,36.2443,-80.8488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,271,Wise Man Brewing,826 Angelos Bros Ave,Winston Salem,NC,27101,1524.0,Michael D,Beverly,Manager,...,NaN,NaN,Private,Forsyth,14,5870000.0,WISEMANBREWING.COM,2017,36.1049,-80.2453
272,272,Wooden Robot Brewery LLC,1440 S Tryon St # 110,Charlotte,NC,28203,4263.0,Joshua,Patton,CEO,...,NaN,NaN,Private,Mecklenburg,14,7049000.0,WOODENROBOTBREWERY.COM,2015,35.2168,-80.8567
273,273,Yesteryears Brewery,300 E Main St # C,Carrboro,NC,27510,2359.0,David,Larsen,Manager,...,NaN,NaN,Private,Orange,14,21819000.0,None,2016,35.91,-79.0688
274,274,York Chester Brewing Co,1500 River Dr # 250,Belmont,NC,28012,3587.0,None,None,None,...,NaN,NaN,Private,Gaston,14,20830000.0,YORKCHESTERBREW.COM,2017,35.2367,-81.0156


In [6]:
# get reviews
search_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

#hit google places api
for index, row in brew_df.iterrows():
    params = {
        "key": gkey,
        "query": f"brewery",
        "location":f"{row.latitude},{row.longitude}"
    }
    
    response = requests.get(search_url, params=params)
    place = response.json()['results'][0]
        
    try:
        brew_df.at[index, 'google_rating'] = place['rating']
        brew_df.at[index, 'google_numratings'] = place['user_ratings_total']

    except:
        pass

In [11]:
brew_df = brew_df.rename(columns={'index':'id'})
brew_df

,id,CompanyName,street,City,State,ZipCode,ZipPlus,ContactFirstName,ContactLastName,ContactTitle,...,BusinessType,County,Employee,AnnualSales,URL,YearEstablished,latitude,longitude,google_rating,google_numratings
0,0,217 Brew Works,217 South St S,Wilson,NC,27893,4911.0,Tom,Curran,Owner,...,Private,Wilson,14,10453000.0,217BREWWORKS.COM,2016,35.7229,-77.9121,4.6,83.0
1,1,All Sevens Brewing,777 Haywood Rd,Asheville,NC,28806,3132.0,None,None,None,...,Private,Buncombe,23,34511000.0,None,2018,35.5787,-82.5943,4.8,4.0
2,2,Altamont Brewing Co,1050 Haywood Rd,Asheville,NC,28806,2639.0,None,None,None,...,Private,Buncombe,23,34511000.0,None,2017,35.5815,-82.6044,4.3,169.0
3,3,Andrews Brewing Co,1026 Main St,Andrews,NC,28901,NaN,None,None,None,...,Private,Cherokee,14,17916000.0,None,2018,35.2008,-83.8244,4.7,146.0
4,4,Angry Troll Brewing,222 E Main St # U2,Elkin,NC,28621,3493.0,Cicely,Mc Culloch,Owner,...,Private,Surry,12,16648000.0,ANGRYTROLLBREWING.COM,2015,36.2443,-80.8488,4.3,281.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,271,Wise Man Brewing,826 Angelos Bros Ave,Winston Salem,NC,27101,1524.0,Michael D,Beverly,Manager,...,Private,Forsyth,14,5870000.0,WISEMANBREWING.COM,2017,36.1049,-80.2453,4.7,302.0
272,272,Wooden Robot Brewery LLC,1440 S Tryon St # 110,Charlotte,NC,28203,4263.0,Joshua,Patton,CEO,...,Private,Mecklenburg,14,7049000.0,WOODENROBOTBREWERY.COM,2015,35.2168,-80.8567,4.6,530.0
273,273,Yesteryears Brewery,300 E Main St # C,Carrboro,NC,27510,2359.0,David,Larsen,Manager,...,Private,Orange,14,21819000.0,None,2016,35.91,-79.0688,4.6,88.0
274,274,York Chester Brewing Co,1500 River Dr # 250,Belmont,NC,28012,3587.0,None,None,None,...,Private,Gaston,14,20830000.0,YORKCHESTERBREW.COM,2017,35.2367,-81.0156,4.9,56.0


In [12]:
# commit back to sqlite database
brew_df.to_sql('business', con=engine, if_exists='replace', index=False)

In [13]:
#test sqlite data
engine.execute("SELECT * FROM business").fetchall()[0]

(0, '217 Brew Works', '217 South St S', 'Wilson', 'NC', 27893, 4911.0, 'Tom', 'Curran', 'Owner', 2529916958, 'Brewers (mfrs)', 312120, None, None, None, None, 'Private', 'Wilson', 14, 10453000.0, '217BREWWORKS.COM', 2016, 35.7229387, -77.912148, 4.6, 83.0)